In [1]:
BRANCH='prompt-learning-documentation'

In [2]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
#!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

'\nYou can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.\n\nInstructions for setting up Colab are as follows:\n1. Open a new Python 3 notebook.\n2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)\n3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)\n4. Run this cell to set up dependencies.\n'

# Introduction

In this notebook we demonstrate how to use p-tunining and prompt tuning within NeMo-Megatron. Both methods are parameter efficient alternatives to fine-tuning pretrained language models. Our NeMo implementation makes it possible to use one pretrained GPT model on many downstream tasks without needing to tune the model's full set of parameters. It also allows for adding new tasks to your model without overwriting or disrupting previous tasks on which the model has already been p-tuned/prompt-tuned. Because the original model parameters are frozen and never altered by either method, p-tuning/prompt-tuning also avoid cartographic forgetting issues often encountered when fine-tuning models. 

- Our prompt tuning implementation is based off Lester et. al’s EMNLP 2021 paper [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/abs/2104.08691)

- Our p-tuning implementation is based off Liu et al's paper [GPT Understands, Too](https://arxiv.org/abs/2103.10385).

- Usage examples and API documentation can be found in [our user docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/megatron.html#prompt-tuning). 

<img src="images/prompt_learning_forward_pass.png" alt="Prompt Learning Forward Pass"/>


# Techincal Overview
Instead of selecting discrete text prompts in a manual or automated fashion, prompt-tuning and p-tuning utilize virtual prompt tokens that can be optimized via gradient decent. The only difference between prompt-tuning and p-tuning within NeMo-Megatron is the architecture used to tune the soft prompt tokens during training. 

### Terminology
We will be using the terms `continuous`, `soft`, and `virtual` interchangeably to refer to tokens inserted into the model prompt that have *no* concrete mapping to strings or characters within the model’s vocabulary. These virtual tokens exist in contrast to `discrete`, `hard`, or `real` tokens that *do* make up the model’s vocabulary.  Virtual tokens are purely 1D vectors with dimensionality equal to that of each real token embedding matching the `hidden_size` hyperparameter. In training and inference, continuous token embeddings are inserted amoung discrete token embeddings according to a template you provide in the model's config. We will show how to do this in the code below. 

When referring to p-tuning and prompt tuning together, we will be using the phrase `prompt learning` for simplicity.


### Prompt-Tuning
When prompt-tuning a pretrained GPT model, soft prompt embeddings are initialized as a 2D matrix of size `total_virtual_tokens X hidden_size`. Each task the model is prompt-tuned to perform has its own 2D embedding matrix associated with it. Tasks do not share any parameters during traning or inference. All GPT model parameters are frozen and only the embedding parameters for each task are updated during training. 

In prompt tuning you can specify how the embeddings are initialized for each task. You can either 

1. Initialize embedding parameters according to some random distribution
2. Initialize embedding parameters from existing vocabulary embeddings (recommended)

If you choose to initialize virtual token embeddings from existing embedding weights, you can provide the string of words you want to use for initialization in the model's config. This string will be tokenized and tiled or truncated to match the specified number of virtual tokens you would like to use (`total_virtual_tokens`). Vocab embeddings are copied and used to initialize the soft prompt embedding matrix for each task. The vocab embeddings themselves are not updated or changed during prompt tuning. 


### P-Tuning
When p-tuning, an LSTM model is used to predict virtual token embeddings. LSTM parameters are randomly initialized at the start of p-tuning. All GPT model parameters are frozen, and only the LSTM weights are updated at each training step. LSTM parameters are shared between all tasks that are p-tuned at the same time, but the LSTM model outputs unique virtual token embeddings for each task. The virtual tokens predicted by the LSTM are inserted among the discrete token input in the exact same manner as with prompt-tuning. You still specify the number of virtual tokens you want to use by setting `total_virtual_tokens` and each virtual token embedding is still a 1D vector of size `hidden_size`.


# The Best of Both
A single pretrained GPT model can use both p-tuning and prompt-tuning. While you must decide to use either p-tuning or prompt-tuning for each task you want your model to perform, you can p-tune your model on a set of tasks *A*, then prompt tune your same model on a different set of tasks *B* and run inference on tasks from both *A* and *B* at the same time. During prompt-tuning or p-tuning, tasks tuned at the same time must use the same number of virtual tokens. During inference, tasks using differing amounts of virtual tokens can be run at the same time. 

P-tuning usually requires fewer virtual tokens per task to achieve good results but uses a higher number of parameters (~ 6% original model params) compared to prompt-tuning (~ .06% of original model params). Because p-tuning shares parameters between tasks during training, p-tuning on many very different tasks at once might perform worse than prompt tuning, which tunes a distinct set of parameters for each task. For the same reason, p-tuning your model on multiple tasks that are similar might allow your model to share insight between tasks.


With all that covered, let's get started!


In [3]:
import os
import wget 
import pathlib

# Tasks and Datasets
We will be using p-tuning and prompt tuning to teach our GPT model to do 3 tasks: **Sentiment Analysis**, **Question Answering** and **Intent and Slot Classification**.

We will use [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for our sentiment analysis task, [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) for question answering task, and the [Assistent Benchmarking Dataset](https://github.com/xliuhw/NLU-Evaluation-Data) for intent and slot classification. 

- The [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) contains the sentiments for financial news headlines from the perspective of a retail investor. Further details about the dataset can be found in Malo et. al's "[Good Debt or Bad Debt: Detecting Semantic Orientations in Economic Texts](https://arxiv.org/abs/1307.5336)"


- [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) is a reading comprehension dataset, consisting of questions posed by crowd workers on a set of Wikipedia articles, where the answer to every question is a segment of text. More information on [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) can be found on their website or in their paper by Rajpurkar et. al "[Know What You Don’t Know: Unanswerable Questions for SQuAD](https://arxiv.org/pdf/1806.03822.pdf)".


- The [Assistent Benchmarking Dataset](https://github.com/xliuhw/NLU-Evaluation-Data) is a natural language dataset for in home human-robot interaction. Details on the dataset can be found in Liu et. al's "[Benchmarking Natural Language Understanding Services for building Conversational Agents](https://arxiv.org/abs/1903.05566)"

Each of these tasks require different types of natural language understanding and lie in different domains. We will demonstrate how to use one model to perform all of them.

# Data Preparation

The prompt learning dataset loader accepts a list of json/dictionary objects or a list of json file names where each json file contains a collection of json objects. Each json object must include the field `taskname` which is a string identifier for the task the data example corresponds to. They should also include one or more fields corresponding to different sections of the discrete text prompt. The input data might look like:

```
[
    {"taskname": "squad", "context": [CONTEXT_PARAGRAPH_TEXT1], "question": [QUESTION_TEXT1], "answer": [ANSWER_TEXT1]},
    {"taskname": "squad", "context": [CONTEXT_PARAGRAPH_TEXT2], "question": [QUESTION_TEXT2], "answer": [ANSWER_TEXT2]},
    {"taskname": "intent_and_slot", "utterance": [UTTERANCE_TEXT1], "label": [INTENT_TEXT1][SLOT_TEXT1]},
    {"taskname": "intent_and_slot", "utterance": [UTTERANCE_TEXT2], "label": [INTENT_TEXT2][SLOT_TEXT2]},
    {"taskname": "sentiment", "sentence": [SENTENCE_TEXT1], "label": [SENTIMENT_LABEL1]},
    {"taskname": "sentiment", "sentence": [SENTENCE_TEXT2], "label": [SENTIMENT_LABEL2]},
]
```

These additional fields can be unlimited in number and will be used to help map different parts of the discrete text input to a prompt template that you define. We will show how this mapping works and how to construct your prompt template in the `Prompt Formatting` section. 

In [4]:
# You can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "data"
NEMO_DIR = '.'

os.makedirs(DATA_DIR, exist_ok=True)


For each dataset we have preprocessing scripts pre-written in NeMo's example directory located in `examples/nlp`. Let's download those now. 

In [5]:
# download the preprocessing scripts from github for the purpose of this tutorial
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/data/prompt_learning_financial_phrase_bank_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/data/prompt_learning_squad_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/prompt_learning_assistant_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/assistant_utils.py', NEMO_DIR)

100% [............................................................] 5619 / 5619

'./assistant_utils.py'

Now let's down load and process each dataset.

###  Financial PhraseBank Dataset

In [6]:
# Download the financial phrase bank dataset
!wget https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip
!unzip FinancialPhraseBank-v10.zip -d {DATA_DIR}

--2022-04-29 15:57:24--  https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip
Resolving www.researchgate.net (www.researchgate.net)... 104.17.33.105, 104.17.32.105, 2606:4700::6811:2169, ...
Connecting to www.researchgate.net (www.researchgate.net)|104.17.33.105|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.researchgate.net/profile/Pekka-Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip [following]
--2022-04-29 15:57:25--  https://www.researchgate.net/profile/Pekka-Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip
Reusing existing connection to www.researchgate.net:443.
HTTP request sent, awaiting response... 200 OK
Length: 681890 (666K) [application/zip]
Saving to: ‘FinancialPhraseBank-v10.zip’

FinancialPhraseBank 

In [7]:
# What the financial phrase bank dataset looks like before processing
SENTIMENT_DIR = os.path.join(DATA_DIR, "FinancialPhraseBank-v1.0")
!head -4 $SENTIMENT_DIR/Sentences_AllAgree.txt

In [8]:
# Preprocess financial phrase bank dataset
!python $NEMO_DIR/prompt_learning_financial_phrase_bank_preprocessing.py

Saving train split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl
100%|███████████████████████████████████| 1811/1811 [00:00<00:00, 285162.91it/s]
Saving val split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_val.jsonl
100%|█████████████████████████████████████| 226/226 [00:00<00:00, 275957.12it/s]
Saving test split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_test.jsonl
100%|█████████████████████████████████████| 227/227 [00:00<00:00, 306340.74it/s]


In [9]:
# What the financial phrase bank dataset looks like after processing
!head -4 $SENTIMENT_DIR/financial_phrase_bank_train.jsonl

{"taskname": "sentiment", "sentence": "The new organization consists of two business units : Charging & Messaging and Finance & Administration .", "label": "neutral"}
{"taskname": "sentiment", "sentence": "Danske Bank is Denmark 's largest bank with 3.5 million customers .", "label": "neutral"}
{"taskname": "sentiment", "sentence": "The total value of the deliveries is some EUR65m .", "label": "neutral"}
{"taskname": "sentiment", "sentence": "In future , the plant will focus on the production of flange profiles for wind farm towers .", "label": "neutral"}


Our financial phrase bank preprocessing script converted the raw text file of sentences and labels into three `.jsonl` files for training, validation, and testing. Each line in the files contains a json object with the fields `taskname`, `sentiment`,`sentence`, and `label`. You can inspect the preprocessing script and play with different arguments for the script by looking at and running `prompt_learning_financial_phrase_bank_preprocessing.py` which should currently be downloaded in `NEMO_DIR`. It is also located at `examples/nlp/text_classification/data/prompt_learning_financial_phrase_bank_preprocessing.py` in the NeMo repo.

By default 80% of the data was randomly selected for the training set, 10% for the validation set, and 10% for the test set. We only used training examples with 100% agreement from labelers on the correct sentiment label. This data is from `Sentences_AllAgree.txt`. This should result in `1811` training examples, `226` validation examples, and `227` examples for testing. The `label` field was removed from test examples. 

If you want to try using more data, you can combine the `Sentences_AllAgree.txt` with any of the `Sentences_75Agree.txt`, `Sentences_66Agree.txt` and/or `Sentences_50Agree.txt` by creating a new cell below and running:

```
!cat {SENTIMENT_DIR}/Sentences_AllAgree.txt {SENTIMENT_DIR}/Sentences_75Agree.txt >> {SENTIMENT_DIR}/combined_data.txt
!python $NEMO_DIR/prompt_learning_financial_phrase_bank_preprocessing.py --file-name combined_data.txt
```


### SQuAD Dataset

In [10]:
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")
os.makedirs(SQUAD_DIR, exist_ok=True)

# Download the SQuAD dataset
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!mv train-v2.0.json {SQUAD_DIR}

--2022-04-29 15:57:27--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   107MB/s    in 0.4s    

2022-04-29 15:57:27 (107 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [11]:
# Preprocess squad data
!python $NEMO_DIR/prompt_learning_squad_preprocessing.py --data-dir {SQUAD_DIR}

Saving train split to data/SQuAD/squad_train.jsonl
100%|█████████████████████████████████| 69125/69125 [00:00<00:00, 170321.58it/s]
Saving val split to data/SQuAD/squad_val.jsonl
100%|███████████████████████████████████| 8952/8952 [00:00<00:00, 191309.76it/s]
Saving test split to data/SQuAD/squad_test.jsonl
100%|███████████████████████████████████| 8744/8744 [00:00<00:00, 196955.02it/s]


In [12]:
# What the squad dataset looks like after processing
!head -4 $SQUAD_DIR/squad_train.jsonl

{"taskname": "squad", "context": "The national bird of Saint Helena is the Saint Helena plover, known locally as the wirebird. It appears on the coat of arms of Saint Helena and on the flag.", "question": "What is the national bird of Saint Helena?", "answer": "the Saint Helena plover"}
{"taskname": "squad", "context": "Other than by appearance, \"Coloureds\" can usually be distinguished from \"Blacks\" by language. Most speak Afrikaans or English as a first language, as opposed to Bantu languages such as Zulu or Xhosa. They also tend to have more European-sounding names than Bantu names.", "question": "What type of names do \"blacks\" have?", "answer": "Bantu names"}
{"taskname": "squad", "context": "The history of the Ottoman Empire during World War I began with the Ottoman engagement in the Middle Eastern theatre. There were several important Ottoman victories in the early years of the war, such as the Battle of Gallipoli and the Siege of Kut. The Arab Revolt which began in 1916 tur

We made a `.jsonl` file for each of the train, validation, and testing splits of the squad data. Every `.jsonl` file contains json objects with the fields `taskname`, `context`, `question`, and `answer`. The preprocessing script is called `prompt_learning_squad_preprocessing.py`. It should be in your `NEMO_DIR` and at `examples/nlp/question_answering/data/prompt_learning_squad_preprocessing.py` in the NeMo repo. 

The SQuAD dataset consists of various topics like `Beyoncé`, `IPod`, and `Symbiosis`. Each topic has several paragraphs associated with it, and each paragraph has several questions and answers related to it. When we separated the train/validation/test splits, we separated them on the topic level. For example, if the training set contains paragraphs and questions about the topic `Beyoncé`, neither the validation nor test sets will contain any questions on this topic. All questions about a certain topic are isolated to one split of the data. 

Like the Financial PhraseBank Dataset, we randomly selected 80% of the questions for training, 10% for validation, and 10% for test. This resulted in `69125` test examples, `8952` validation examples, and `8744` testing examples. The `answer` field was removed from test examples.

### Assistent Dataset

In [13]:
ASSISTENT_DIR = os.path.join(DATA_DIR, "assistent")
os.makedirs(ASSISTENT_DIR, exist_ok=True)

# Download the assisent dataset
!wget https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip
!unzip master.zip -d {ASSISTENT_DIR}

--2022-04-29 15:57:29--  https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/xliuhw/NLU-Evaluation-Data/zip/refs/heads/master [following]
--2022-04-29 15:57:29--  https://codeload.github.com/xliuhw/NLU-Evaluation-Data/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [             <=>    ]  20.53M  8.09MB/s    in 2.5s    

2022-04-29 15:57:32 (8.09 MB/s) - ‘master.zip’ saved [21527506]

Archive:  master.zip
f6071b496b17d71e6eb43f543af0707f4ff30557
   creating: data/assistent/NLU-Evaluation-Data-master/
 

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_affirm.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_commandstop.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_confirm.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_dontcare.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/trainset/general_explain.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_2/trainset/email_query.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_2/trainset/email_querycontact.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_2/trainset/email_sendemail.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_2/trainset/general_affirm.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_2/trainset/general_commandstop.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGe

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_5/testset/annotated/iot_wemo_on/iot_wemo_on_anno_20180322130134666.json  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_5/testset/annotated/lists_createoradd/
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_5/testset/annotated/lists_createoradd/lists_createoradd_anno_20180322130134738.json  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_5/testset/annotated/lists_query/
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_5/testset/annotated/lists_query/l

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_7/trainset/lists_query.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_7/trainset/lists_remove.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_7/trainset/music_likeness.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_7/trainset/music_query.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_7/trainset/music_settings.csv  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-1

...

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_5/merged/intents/transport_traffic.json  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_5/merged/intents/weather_query.json  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_5/merged_apiai_kfold_5.zip  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_6/
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_6/merged-apiai-kfold-6.zip  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/Cross

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_8/merged/intents/email_query.json  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_8/merged/intents/email_querycontact.json  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_8/merged/intents/email_sendemail.json  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_8/merged/intents/general_affirm.json  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_8/merged/intents/general_commandstop.json  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidati

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_2/merged/Luis_KFold_2_2018_03_22-13_01_25_169_20180327170202385.json  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_3/
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_3/merged/
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_3/merged/Luis_KFold_3_2018_03_22-13_01_25_169_20180327170202385.json  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_4/
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_4/merged/RasaNlu_Merged_KFold_4_2018_03_22-13_01_25_169.json  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_4/mergedTestset/
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_4/mergedTestset/RasaNluTestset_Merged_2018_03_22-13_01_25_169.json  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_4/rasaTrainConfig/
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_4/rasaTrainConfig/configLatest_KFold_4_mitie_2018_03_22-13_01_25_169.json  
  inflating: dat

  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_9/mergedTestset/RasaNluTestset_Merged_2018_03_22-13_01_25_169.json  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_9/rasaTrainConfig/
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_9/rasaTrainConfig/configLatest_KFold_9_mitie_2018_03_22-13_01_25_169.json  
  inflating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_9/rasaTrainConfig/configLatest_KFold_9_spacy_sklearn_2018_03_22-13_01_25_169.json  
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/
   creating: data/assistent/NLU-Evaluation-Data-master/CrossValidation/ou

In [14]:
# Process virtual assistent intent and slot classification data
!python $NEMO_DIR/prompt_learning_assistant_preprocessing.py

[NeMo W 2022-04-29 15:57:37 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2022-04-29 15:57:37 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-04-29 15:57:37 assistant_utils:127] Processing assistant commands dataset and store at data/assistant/nemo-format
[NeMo I 2022-04-29 15:57:37 assistant_utils:31] Copying files to input folder: data/assistant/NLU-Evaluation-Data-master/dataset
Traceback (most recent call last):
  File "./prompt_learning_assistant_preprocessing.py", line 184, in <module>
    main()
  File "./prompt_learning_assistant_preprocessing.py

In [15]:
!head -5 $ASSISTENT_DIR/assistent_train.jsonl

head: cannot open 'data/assistent/assistent_train.jsonl' for reading: No such file or directory


For the virtual assistent dataset, there are a set of 64 possible intents:

In [16]:
!echo 'Intents: ' $(wc -l < {ASSISTENT_DIR}/nemo-format/dict.intents.csv)
!cat {ASSISTENT_DIR}/nemo-format/dict.intents.csv

/bin/bash: data/assistent/nemo-format/dict.intents.csv: No such file or directory
Intents: 
cat: data/assistent/nemo-format/dict.intents.csv: No such file or directory


and 55 types of slots:

In [17]:
# print all slots from the NeMo format slot dictionary
!echo 'Slots: ' $(wc -l < {ASSISTENT_DIR}/nemo-format/dict.slots.csv)
!cat {ASSISTENT_DIR}/nemo-format/dict.slots.csv

/bin/bash: data/assistent/nemo-format/dict.slots.csv: No such file or directory
Slots: 
cat: data/assistent/nemo-format/dict.slots.csv: No such file or directory


Each slot label consists of the slot type followed by specific text from the utterance corresponding to that slot type in parentheses. For example, the utterance `"tell my facebook group that i've arrived"` has the intent label `social_post` and the slot label `media_type(facebook)`. Utterances each have one intent label and zero or more slot labels. In cases where there is no slot label, our GPT model should predict the word `None`. 

Json objects for each training example contain three fields: `taskname`, `utterance`, and `label`. For this dataset, our preprocessing scipt formatted our intent and slot labels to look like `"\nIntent: transport_taxi\nSlots: transport_agency(golden taxi), time(seven pm), date(today)"`. With newline characters (\n) separating intent and slot labels. Our train jsonl file has `9960` training examples. Our validation and test jsonl files each have `538` training examples. Test examples do not have the `label` field. 

# Model Config Setup

Now we will begin setting up the conifg file used for prompt/p-tuning our GPT models! GPT Prompt learning within NeMo uses a class called `MegatronGPTPromptLearningModel` which has its own config file. We will start by loading an example prompt learning config file, then make changes to it to fit our tasks and training plans. 

In [18]:
from omegaconf import OmegaConf

CONFIG_DIR = os.path.join(NEMO_DIR, "conf")
os.makedirs(CONFIG_DIR, exist_ok=True)

# Download the example config file
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/megatron_gpt_prompt_learning_config.yaml', CONFIG_DIR)

# Load the example config file so we can start editing it
CONFIG_PATH = os.path.join(CONFIG_DIR, "megatron_gpt_prompt_learning_config.yaml")
config = OmegaConf.load(CONFIG_PATH)

100% [............................................................] 4405 / 4405

First let's set the datasets we've created in the config. We are going to start by p-tuning a GPT model on the **Sentiment Analysis** and **Intent and Slot Classification** tasks. We will set only those two datasets in the config file right now. 

In [19]:
config.model.data.train_ds = [f"{SENTIMENT_DIR}/financial_phrase_bank_train.jsonl", f"{ASSISTENT_DIR}/assistent_train.jsonl"]
config.model.data.validation_ds = [f"{SENTIMENT_DIR}/financial_phrase_bank_val.jsonl", f"{ASSISTENT_DIR}/assistent_val.jsonl"]

The `MegatronGPTPromptLearningModel` class expects datasets to be a list of `.json` or `.jsonl` file paths. You can give it multiple datasets at once like we did above.  

### Prompt Formatting
Now that we have our datasets set, lets define what we want the prompt for each task to look like. 

To customize different prompts for different tasks, we simply need to specify the prompt task template in the config file. The virtual token markers `<|VIRTUAL_PROMPT_#|>` signify where you want virtual tokens to be placed in the template string. `<|VIRTUAL_PROMPT_0|>`, `<|VIRTUAL_PROMPT_1|>`, and `<|VIRTUAL_PROMPT_2|>` indicate where a number of virtual tokens matching the values given at `virtual_token_splits[0]`, `virtual_token_splits[1]` and `virtual_token_splits[2]` will be placed. The other variable fields `{var}` refer to the fields in the data json. 

For example, given: 

1. the data json **{"sentence1": "And he said, Mama, I'm home.", "sentence2": "He didn't say a word."}**


2. virtual token splits set to `virtual_token_splits = [3, 3, 3]` 


3. a prompt template set to `prompt_template = "<|VIRTUAL_PROMPT_0|> Hypothesis: [sentence1], <|VIRTUAL_PROMPT_1|> Premise: [sentence2] <|VIRTUAL_PROMPT_2|> Answer:"`

the input will be translated into **<span style="color:red">VVV</span> Hypothesis: And he said, Mama, I'm home.<span style="color:red">VVV</span> Premise: He didn't say a word.<span style="color:red">VVV</span> Answer:**, where <span style="color:red">VVV</span> are three virtual tokens.

Let's configure a template for our **Sentiment Analysis** and **Intent and Slot Classification** tasks:


In [20]:
config.model.task_templates = [
    {
        "taskname": "sentiment",
        "prompt_template": "<|VIRTUAL_PROMPT_0|> {sentence} sentiment: {label}",
        "total_virtual_tokens": 10,
        "virtual_token_splits": [10],
        "truncate_field": None
    },
    {
        "taskname": "intent_and_slot",
        "prompt_template": "<|VIRTUAL_PROMPT_0|> Predict intent and slot <|VIRTUAL_PROMPT_1|> :\n{utterance}{label}",
        "total_virtual_tokens": 10,
        "virtual_token_splits": [7, 3],
        "truncate_field": None
    }
]

Note each `task_template` item has 5 fields. 

- **`prompt_template`** is a string showing the model where to place virtual tokens and how to map dataset json fields to where they belong in the model prompt. 


- **`taskname`** refers to the same `taskname` in the dataset json objects. 


- **`truncate_field`** specifies which field in the data json to truncate if the length of the input exceeds the maximum sequence length of the model. If `truncate_field` is set to `None`, examples that are too long are simply dropped from the dataset.


- **`total_virtual_tokens`** specifies the total number of virtual tokens that will be inserted into the model prompt.


- **`virtual_token_splits`** specifies the number of virtual tokens that belong at each `<|VIRTUAL_PROMPT_#|>` marker. `virtual_token_splits` values should add up to `total_virtual_tokens`. The number of `virtual_token_splits` should match the number of `<|VIRTUAL_PROMPT_#|>` markers. 

### Setting New Tasks
After you p-tune your model this time, you can always go back and p-tune or prompt-tune your model on more tasks without over writting the virtual prompts who've trained this time. You can also use a different number of `total_virtual_tokens` between each training session as long as tasks ptuned or prompt tuned at the same time have the same number of `total_virtual_tokens`. For this reason, when you ptune on a new task, you need to tell your model which of your tasks are new and which ones already exist (and thus you don't want to tune them). 

You do this by setting the `new_tasks` and `existing_tasks` values in the config file. Because we are ptuning a model with no existing tasks, you should set `existing_tasks=[]` and `new_tasks=["sentiment", "intent_and_slot"]` as follows:

In [21]:
config.model.existing_tasks = []
config.model.new_tasks = ["sentiment", "intent_and_slot"]

After ptuning and/or prompt tuning is complete, you can run inference on all tasks at the same time, regradless of their `total_virtual_tokens` value.

### Setting The Pre-Trained GPT Model
We still need to set which GPT model we want to p-tune/prompt tune. Prompt learning methods work best with large GPT language models (5B or above), but the purposes of this tutorial, we are going to download a 345M parameter GPT model from NVIDIA NGC and convert it to `.nemo` format. We will be using the conversion script located at `examples/nlp/language_model/megatron_lm_ckpt_to_nemo.py`. The next cell can be skipped and/or removed after `.nemo` files are added to NGC for direct download without conversion. 

In [22]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
MegatronGPTModel.list_available_models()

[NeMo W 2022-04-29 15:57:42 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2022-04-29 15:57:42 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[PretrainedModelInfo(
 	pretrained_model_name=megatron_gpt_345m,
 	description=345M parameter GPT generative Megatron model.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo
 )]

If we wanted to use the GPT model class directly, we could download the model by calling: 
`gpt_model = MegatronGPTModel.from_pretrained(model_name="megatron_gpt_345m").cuda()`

In [23]:
gpt_model = MegatronGPTModel.from_pretrained(model_name="megatron_gpt_345m").cuda()

[NeMo I 2022-04-29 15:57:43 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.9.0rc0/megatron_gpt_345m/5c8c7aba34ae4c5906609360fcd22aaf/megatron_gpt_345m.nemo.
[NeMo I 2022-04-29 15:57:43 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.9.0rc0/megatron_gpt_345m/5c8c7aba34ae4c5906609360fcd22aaf/megatron_gpt_345m.nemo
[NeMo I 2022-04-29 15:57:43 common:789] Instantiating model from pre-trained checkpoint


AttributeError: 'NoneType' object has no attribute 'accumulate_grad_batches'

In [ ]:
# Download the model from NGC
!wget

In [ ]:
# Download the model and its needed files
CONF_DIR = os.path.join(NEMO_DIR, "conf")
GPT_MODEL_CONF = "megatron_gpt_config.yaml"
os.makedirs(CONF_DIR, exist_ok=True)

ngc_gpt_file = "megatron_lm_345m_v0.0.zip"
vocab_file = "gpt2-vocab.json"
merge_file = "gpt2-merge.txt"
checkpoint_filename = "model_optim_rng.pt"
nemo_gpt_file = "gpt_344m.nemo"

if not pathlib.Path(ngc_gpt_file).exists():
    ! wget --content-disposition https://api.ngc.nvidia.com/v2/models/nvidia/megatron_lm_345m/versions/v0.0/zip -O {NEMO_DIR}/{ngc_gpt_file}
    ! unzip -o {NEMO_DIR}/{ngc_gpt_file}
    ! wget https://s3.amazonaws.com/models.huggingface.co/bert/$vocab_file -O {NEMO_DIR}/{vocab_file}
    ! wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt -O {NEMO_DIR}/{merge_file}

# Download the model config and conversion scipt
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/' + GPT_MODEL_CONF, CONF_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/megatron_lm_ckpt_to_nemo.py', NEMO_DIR)

# Set 345M specific params in the downloaded config
gpt_config = OmegaConf.load(os.path.join(CONF_DIR, GPT_MODEL_CONF))
gpt_config.model.num_layers = 24
gpt_config.model.hidden_size = 1024
gpt_config.model.ffn_hidden_size = 4096
gpt_config.model.num_attention_heads = 16
gpt_config.model.tokenizer.vocab_file = vocab_file
gpt_config.model.tokenizer.merge_file = merge_file
gpt_config.model.tensor_model_parallel_size = 1
gpt_config.model.data.data_prefix = ''
gpt_config.model.max_position_embeddings = 1024
gpt_config.model.data.seq_length = 1024
gpt_config.model.encoder_seq_length = 1024
gpt_config.cfg = {}
gpt_config.cfg.cfg = gpt_config.model

with open(f"{NEMO_DIR}/hparams.yaml", "w") as f:
    f.write(OmegaConf.to_yaml(gpt_config.cfg))
    
# Run the conversion script
!python -m torch.distributed.run --nproc_per_node=1 megatron_lm_ckpt_to_nemo.py --checkpoint_folder=$NEMO_DIR/release/mp_rank_00/ --checkpoint_name=$checkpoint_filename --hparams_file=$NEMO_DIR/hparams.yaml --nemo_file_path=$NEMO_DIR/$nemo_gpt_file --model_type=gpt --tensor_model_parallel_size=1

Now that we have a `.nemo` GPT file to work with. We need to add its path in our prompt learning config. 

In [ ]:
config.model.language_model_path = nemo_gpt_file

We can also set where we want the final prompt tuned model to be saved by setting `model.nemo_path`. By default the tuned prompt learning model will be saved in your current working directory to a `.nemo` file with the same name as your experiment (`config.name`). Let's change the save name to be `multitask_p_tuned_gpt.nemo`. **Your model path must end in `.nemo`.**

In [ ]:
config.model.nemo_path = "multitask_p_tuned_gpt.nemo"

### Setting P-Tuning Specific Params
Within the config file, p-tuning and prompt-tuning each have a couple of hyperparameters specific to them. We first need to tell the model that we want to do p-tuning, not prompt-tuning. To do this, we set the **`model.virtual_prompt_style`** hyperparameter like this:

In [ ]:
config.model.virtual_prompt_style = 'p-tuning'

Then we can set the 2 p-tuning specific parameters. Reminder, pp-tuning uses an LSTM prompt encoder to predict virtual tokens. 

- **`p_tuning.dropout`** the LSTM prompt encoder dropout probability 
- **`p_tuning.num_layers`** the number of LSTM layers you want your p-tuning prompt encoder to have


In [ ]:
config.model.p_tuning.dropout = 0.0
config.model.p_tuning.num_layers = 2

Let's have a look at all the values we've set in the model config. You can change any of these values in the same manner we've been using above. 

In [ ]:
print(OmegaConf.to_yaml(config.model))

# Building the PyTorch Lightning Trainer
NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [ ]:
import torch
import pytorch_lightning as pl
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPPlugin
from pytorch_lightning.plugins.environments.torchelastic_environment import TorchElasticEnvironment

# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.accelerator = accelerator
config.trainer.devices = 1
config.trainer.max_epochs = 1
config.trainer.val_check_interval = 1.0

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
config.trainer.strategy = None

# setup cluster environment parameters"
# use torch elastic cluster environment so `create_process_externally` is True
# the launcher is set to None. It will not try to spawn new processes.
# It won't create the misconfiguration error because of the `interactive session`
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'

plugins = [NLPDDPPlugin(find_unused_parameters=False), TorchElasticEnvironment()]
trainer = pl.Trainer(plugins=plugins, **config.trainer)

print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

# Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [ ]:
from nemo.utils.exp_manager import exp_manager

# Set name of the experiment 
config.name = 'sentiment_intent_slot_p_tuning'

# Init the experiment manager and view the exp_dir
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_dir = str(exp_dir)
print(exp_dir)

We can also set learning hyperparameters as follows:

In [ ]:
# Set some of the learning parameters
config.model.optim.lr = 1e-5
config.model.batch_size = 16

# Training (P-Tuning)
The only thing left to do is load up the model and begin p-tuning!

In [ ]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_prompt_learning_model import MegatronGPTPromptLearningModel

model = MegatronGPTPromptLearningModel(cfg=config.model, trainer=trainer)

In [ ]:
trainer.fit(model)

When training completes, prompt tuned virtual tokens from the prompt encoder are automatically moved to a `prompt_table` where all prompt tuned and p-tuned soft prompts are stored. The LSTM `prompt_encoder` is then removed from the model. This allows us to preserve previously p-tuned soft prompts while still maintaining the ability to add new p-tuned or prompt-tuned soft prompts in the future. The `prompt_table` uses the `taskname` as a key to look up the correct virtual tokens for a specified task.

# Inference After P-Tuning
One way to run inference after p-tuning or prompt-tuning your model is to call `model.generate()`. `model.generate()` takes in 

- `inputs` which can be either a list of dictionary objects or `.jsonl` files containing dictionary objects, 
- `length_params`
- `sampling_params`

as arguments. 

If `length_params` and `sampling_params` are set to `None`, the model generates output with a greedy decoding strategy and generates up to `30` new tokens. To see other ways to run inference with your prompt learning model and more details on how to define various inference parameters, visit `examples/nlp/language_modeling/megatron_gpt_eval.py`.

Below are some randomly selected test examples from the sentiment classification and intent and slot classification test files. Notice that the `label` field is dropped from all test examples. The `MegatronPromptLearningDataset` called within `.generate()` automatically leaves fields in the prompt template empty when they are not provided in the data json. 

In [ ]:
test_examples = [
    {"taskname": "intent_and_slot", "utterance": "tell me who will win the next presidential election"},
    {"taskname": "intent_and_slot", "utterance": "please will you check it."},
    {"taskname": "intent_and_slot", "utterance": "is there anything i need to be aware of"},
    {"taskname": "intent_and_slot", "utterance": "set the alarm to seven am for work"},
    {"taskname": "sentiment", "sentence": "The products have a low salt and fat content ."},
    {"taskname": "sentiment", "sentence": "The agreement is valid for four years ."},
    {"taskname": "sentiment", "sentence": "Diluted EPS rose to EUR3 .68 from EUR0 .50 ."},
    {"taskname": "sentiment", "sentence": "The company is well positioned in Brazil and Uruguay ."},
    {"taskname": "sentiment", "sentence": "Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier ."},
]

This allows us to prompt the p-tuned GPT model as follows:

```
"<|VIRTUAL_PROMPT_0|> Predict intent and slot <|VIRTUAL_PROMPT_1|> :\nplease will you check it."
"<|VIRTUAL_PROMPT_0|> Predict intent and slot <|VIRTUAL_PROMPT_1|> :\nset the alarm to seven am for work"
------------------
"<|VIRTUAL_PROMPT_0|> The products have a low salt and fat content . sentiment:"
"<|VIRTUAL_PROMPT_0|> The agreement is valid for four years . sentiment:"

```

With the correct virtual tokens inserted at each `<|VIRTUAL_PROMPT_#|>`

In [ ]:
response = model.generate(inputs=test_examples, length_params=None)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print(result)
    print("-" * 30)